In [1]:
from crocoddyl import *
import pinocchio as pin
import numpy as np

robot = loadKintonArm()
robot.initViewer(loadModel=True)
robot.display(robot.q0)

robot.framesForwardKinematics(robot.q0)

# Create a cost model per the running and terminal action model.
runningCostModel = CostModelSum(robot.model)
terminalCostModel = CostModelSum(robot.model)

In [2]:
target_pos  = np.array([-0.1,-0.05,-0.15])
target_quat = pin.Quaternion(.4, .02, -.5, .7)
target_quat.normalize()

# Plot goal frame
robot.viewer.gui.addXYZaxis('world/framegoal', [1., 0., 0., 1.], .015, 1)
robot.viewer.gui.applyConfiguration('world/framegoal', target_pos.tolist() + [target_quat[0], target_quat[1], target_quat[2], target_quat[3]])
robot.viewer.gui.refresh()

IDX_BASE = robot.model.getFrameId('base_link', pin.FrameType.BODY)
Mbase = robot.data.oMf[IDX_BASE]
robot.viewer.gui.addXYZaxis('world/framebase', [1., 0., 0., 1.], .03, 0.5)
robot.viewer.gui.applyConfiguration('world/framebase', pin.se3ToXYZQUATtuple(Mbase))
robot.viewer.gui.refresh()

IDX_EE = robot.model.getFrameId('link6', pin.FrameType.BODY)
Mee = robot.data.oMf[IDX_EE]
robot.viewer.gui.addXYZaxis('world/frameee', [1., 0., 0., 1.], .03, 0.5)
robot.viewer.gui.applyConfiguration('world/frameee', pin.se3ToXYZQUATtuple(Mee))
robot.viewer.gui.refresh()

In [3]:
# Note that we need to include a cost model (i.e. set of cost functions) in
# order to fully define the action model for our optimal control problem.
# For this particular example, we formulate three running-cost functions:
# goal-tracking cost, state and control regularization; and one terminal-cost:
# goal cost. First, let's create the common cost functions
frameName = 'link6'
state = StatePinocchio(robot.model)
SE3ref = pin.SE3()
SE3ref.translation = target_pos.reshape(3,1)
SE3ref.rotation = target_quat.matrix()

goalTrackingCost = CostModelFramePlacement(robot.model, nu=robot.model.nv, frame=robot.model.getFrameId(frameName), ref=SE3ref)
xRegCost = CostModelState(robot.model, state, ref=state.zero(), nu=robot.model.nv)
uRegCost = CostModelControl(robot.model, nu=robot.model.nv)

# Then let's add the running and terminal cost functions
runningCostModel.addCost(name="pos", weight=1e-3, cost=goalTrackingCost)
runningCostModel.addCost(name="regx", weight=1e-7, cost=xRegCost)
runningCostModel.addCost(name="regu", weight=1e-7, cost=uRegCost)
terminalCostModel.addCost(name="pos", weight=50, cost=goalTrackingCost)

In [4]:
# Next, we need to create an action model for running and terminal knots. The
# forward dynamics (computed using ABA) are implemented
# inside DifferentialActionModelFullyActuated.
runningModel = IntegratedActionModelEuler(DifferentialActionModelFullyActuated(robot.model, runningCostModel))
terminalModel = IntegratedActionModelEuler(DifferentialActionModelFullyActuated(robot.model, terminalCostModel))

# Defining the time duration for running action models and the terminal one
dt = 1e-3
runningModel.timeStep = dt

# For this optimal control problem, we define 250 knots (or running action
# models) plus a terminal knot
T = 250
q0 = [0., 0., 0., 0., 0., 0.]
x0 = np.hstack([q0, np.zeros(robot.model.nv)])
problem = ShootingProblem(x0, [runningModel] * T, terminalModel)

# Creating the DDP solver for this OC problem, defining a logger
ddp = SolverDDP(problem)
# ddp.callback = [CallbackDDPVerbose()]
# ddp.callback.append(CallbackDDPLogger())

# Solving it with the DDP algorithm
ddp.solve()


KeyboardInterrupt: 

In [9]:
from crocoddyl.diagnostic import displayTrajectory

displayTrajectory(robot, ddp.xs, runningModel.timeStep)